# Lecture 9: Spatial Analysis - SOLUTION

**Date:** January 2, 2026

---

In [ ]:
import scanpy as sc
import squidpy as sq

sc.settings.verbosity = 3

## Task 1: Loading Spatial Data (20 points)

In [ ]:
# Load Visium data
adata = sq.datasets.visium_hne_adata()
print(f"Loaded: {adata.n_obs} spots × {adata.n_vars} genes")

# Visualize spatial coordinates
sq.pl.spatial_scatter(adata, color='total_counts', size=1.5)

# Plot gene expression in spatial context
sq.pl.spatial_scatter(adata, color=['Hpca', 'Ttr', 'Plp1'], ncols=3, size=1.5)

print("Spatial data loaded!")

## Task 2: Spatial Neighborhood Analysis (30 points)

In [ ]:
# Build spatial graph
sq.gr.spatial_neighbors(adata, coord_type='generic', n_neighs=6)

# Clustering
sc.pp.normalize_total(adata, inplace=True)
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata, n_top_genes=1000)
sc.pp.pca(adata)
sc.pp.neighbors(adata)
sc.tl.leiden(adata, resolution=0.5)

# Visualize clusters
sq.pl.spatial_scatter(adata, color='leiden', size=1.5)

# Neighborhood enrichment
sq.gr.nhood_enrichment(adata, cluster_key='leiden')
sq.pl.nhood_enrichment(adata, cluster_key='leiden')

print("Spatial neighborhoods analyzed!")

## Task 3: Spatially Variable Genes (30 points)

In [ ]:
# Identify spatially variable genes
sq.gr.spatial_autocorr(adata, mode='moran', n_perms=100, n_jobs=1)

# Top SVGs
svg_results = adata.uns['moranI'].sort_values('I', ascending=False)
print("\nTop 10 spatially variable genes:")
print(svg_results.head(10))

# Visualize top SVGs
top_svgs = svg_results.head(6).index
sq.pl.spatial_scatter(adata, color=top_svgs, ncols=3, size=1.5, cmap='viridis')

# Compare with HVGs
hvgs = set(adata.var_names[adata.var['highly_variable']])
svgs = set(svg_results.head(100).index)
overlap = hvgs.intersection(svgs)
print(f"\nOverlap between HVGs and SVGs: {len(overlap)}")

print("Spatially variable genes identified!")

## Task 4: Spatial Niches (20 points)

In [ ]:
# Define niches based on clusters
adata.obs['niche'] = adata.obs['leiden'].map({
    '0': 'Cortex',
    '1': 'Hippocampus',
    '2': 'White matter',
    '3': 'Ventricle'
}).fillna('Other')

# Visualize niches
sq.pl.spatial_scatter(adata, color='niche', size=1.5, legend_loc='right')

# Niche-specific markers
sc.tl.rank_genes_groups(adata, 'niche', method='wilcoxon')
sc.pl.rank_genes_groups(adata, n_genes=10, sharey=False)

print("Spatial niches characterized!")

---

## Summary

- ✓ Spatial data visualization
- ✓ Spatial neighborhoods
- ✓ Spatially variable genes
- ✓ Spatial niche identification